<div class="alert alert-block alert-danger">

# FIT5196 Task 2 in Assessment 1
    
Student Name: Anushka Jemima

Student ID: 33617457

Student Name: Shruthi Shashidhara Shastry

Student ID: 33684669

Date: 30-08-2024

Environment: Python

Libraries used:
* os (for interacting with the operating system, included in Python xxxx)
* pandas 1.1.0 (for dataframe, installed and imported)
* multiprocessing (for performing processes on multi cores, included in Python 3.6.9 package)
* itertools (for performing operations on iterables)
* nltk 3.5 (Natural Language Toolkit, installed and imported)
* nltk.tokenize (for tokenization, installed and imported)
* nltk.stem (for stemming the tokens, installed and imported)

    </div>

<div class="alert alert-block alert-info">
    
## Table of Contents

</div>

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Input File](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Tokenization](#tokenize) <br>
$\;\;\;\;$[4.2. Whatever else](#whetev) <br>
$\;\;\;\;$[4.3. Genegrate numerical representation](#whetev1) <br>
[5. Writing Output Files](#write) <br>
$\;\;\;\;$[5.1. Vocabulary List](#write-vocab) <br>
$\;\;\;\;$[5.2. Sparse Matrix](#write-sparseMat) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

<div class="alert alert-block alert-success">
    
## 1.  Introduction  <a class="anchor" name="Intro"></a>


This assessment concerns textual data and the aim is to extract data, process them, and transform them into a proper format. The dataset provided is in the format of a json file containing Google Maps review data.

The main objectives of this task are:
1. Load and parse the JSON file containing Google Maps review data
2. Preprocess the review text data
3. Generate a vocabulary list from the processed data
4. Create a sparse matrix representation of the reviews

This notebook presents our approach to accomplish these objectives efficiently and accurately.

<div class="alert alert-block alert-success">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>

In this assessment, any python packages is permitted to be used. The following packages were used to accomplish the related tasks:

* **os:** to interact with the operating system, e.g. navigate through folders to read files
* **re:** to define and use regular expressions
* **pandas:** to work with dataframes
* **multiprocessing:** to perform processes on multi cores for fast performance
* **langid:** for language identification
* **itertools:** For efficient iteration over data structures
* **nltk:** Provides a suite of text processing libraries for tokenization, stemming, and n-gram generation.
* **json:** for parsing JSON data.
* **collections:** for efficient dictionary operations and counting

The NLTK-specific imports (RegexpTokenizer, MWETokenizer, PorterStemmer, ngrams, bigrams, FreqDist)
provide specialized tools for natural language processing tasks such as tokenization, stemming,
and n-gram generation.

These libraries will enable us to efficiently process the Google Maps review data and generate the required outputs.

In [25]:
#install if required
!pip install langid

In [26]:
# Import required libraries
import os
import re
import langid
import pandas as pd
import multiprocessing
from itertools import chain
import nltk
from nltk.probability import *
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import json
from nltk import bigrams, FreqDist
from collections import Counter
from collections import defaultdict


-------------------------------------

<div class="alert alert-block alert-success">
    
## 3.  Examining Input File <a class="anchor" name="examine"></a>

In [27]:
#Mount the goole drive , to access files stored your google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Let's examine what is the content of the file. For this purpose, we load and parse the json file containing review data as seen below.

Make sure you have stored your json file before running the cells below.


In [28]:

# Load the JSON data from the file generated in Task 1
def load_json_file(filename):
  with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)
  return data

# Load the data
#Change the path as per the location of the data
data = load_json_file('task1_103.json')

# Display a sample of the data structure
print("Sample of the data structure:")
print(json.dumps(list(data.items())[0], indent=2))

Sample of the data structure:
[
  "0x8082d8790156d331:0x50e8adef53c39a42",
  {
    "reviews": [
      {
        "user_id": "101312766566645034554",
        "time": "2021-05-16 13:54:55",
        "review_rating": "1",
        "review_text": "seated outside. waited 20 minutes for someone to bring us water and take our order. had to eventually go get someone. waited another 20 minutes for the food and a beer that we had ordered. ordered 5 sushi rolls. each had half an inch of bland over cooked rice surrounding the roll. the insides weren\u2019t much better and i am amazed at how bland the crab, fish and vegetables were. the only explanation is that the ingredients were old and possibly over cooked. will not return unfortunately.",
        "if_pic": "N",
        "pic_dim": [],
        "if_response": "N"
      },
      {
        "user_id": "104762802021190077178",
        "time": "2021-05-01 02:10:05",
        "review_rating": "2",
        "review_text": "we had standard maki and ngiri at g


It is noticed that the file contains:
1. A dictionary where each key is a unique business identifier(GmapID).
2. Each business entry contains a 'reviews' key with a list of review dictionaries.
3. Each review dictionary includes:
   - user_id: Identifier of the reviewer
   - time: Timestamp of the review
   - review_rating: Numerical rating given by the reviewer
   - review_text: The actual text content of the review
   - if_pic: Indicates if the review includes pictures
   - pic_dim: Dimensions of attached pictures, if any
   - if_response: Indicates if the business responded to the review
   


Having parsed the JSON file, the following observations can be made:
1. The dataset contains reviews for multiple businesses.
2. Each business has a varying number of reviews.
3. Reviews include both textual and metadata information.
4. The review texts are the primary data we'll be working with for text processing tasks.


This structure will inform our approach to filtering and processing the reviews in the subsequent steps.



<div class="alert alert-block alert-success">
    
## 4.  Loading and Parsing File <a class="anchor" name="load"></a>

In this section, we will load the JSON data, parse it, and filter the reviews based on our criteria.
We'll focus on businesses with at least 70 valid text reviews.

In [29]:
# Initialize a dictionary to store businesses with at least 70 text reviews
filtered_reviews = {}

# loop over each business in the JSON data
for business_id, business_data in data.items():
    reviews = business_data['reviews']

    # Filter reviews that contain valid text that is text is not none
    text_reviews = [review['review_text'].lower() for review in reviews
                    if review['review_text'] and review['review_text'].lower() != 'none' ]

    # Only keep businesses with at least 70 text reviws
    if len(text_reviews) >= 70:
        filtered_reviews[business_id] = text_reviews



In [30]:
# Print the number of businesses with at least 70 valid reviews
print(f"Number of businesses with at least 70 valid reviews: {len(filtered_reviews)}")

Number of businesses with at least 70 valid reviews: 87


In [31]:
# display a sample of the text reviews
print("Sample of a text review:")
print(next(iter(filtered_reviews.values()))[0])

Sample of a text review:
seated outside. waited 20 minutes for someone to bring us water and take our order. had to eventually go get someone. waited another 20 minutes for the food and a beer that we had ordered. ordered 5 sushi rolls. each had half an inch of bland over cooked rice surrounding the roll. the insides weren’t much better and i am amazed at how bland the crab, fish and vegetables were. the only explanation is that the ingredients were old and possibly over cooked. will not return unfortunately.



Let's examine the dictionary generated. For counting the total number of reviews extracted:

In [32]:
total_reviews = sum(len(reviews) for reviews in filtered_reviews.values())
# Display Total number the text review
print(f"Total number of reviews extracted: {total_reviews}")

average_reviews = total_reviews / len(filtered_reviews)
#Display Average number of text reviews
print(f"Average number of reviews per business: {average_reviews:.2f}")

Total number of reviews extracted: 19851
Average number of reviews per business: 228.17


Observations from the parsing and filtering process:
1. We successfully filtered businesses based on the criterion of having at least 70 valid text reviews.
2. The filtering process removed any empty reviews or reviews with the text 'none'.
3. All review texts were converted to lowercase for consistency in future processing steps.
4. We retained {len(filtered_reviews)} businesses that met our criteria.
5. A total of {total_reviews} reviews were extracted across all filtered businesses.
6. On average, each filtered business has approximately {average_reviews:.2f} reviews.

This filtered dataset will be the basis for our further text processing tasks, including tokenization,
vocabulary generation, and creation of the sparse matrix representation


<div class="alert alert-block alert-warning">
    
### 4.1. Tokenization <a class="anchor" name="tokenize"></a>

Tokenization is a principal step in text processing and producing unigrams. In this section, we will tokenize the reviews and prepare them for further processing.

In [33]:

# Define the regular expression tokenizer
tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:[-.'][a-zA-Z]+)?")

# This function will tokenize each review in the dictionary
def tokenize_review(review):
    return tokenizer.tokenize(review.lower())

# Tokenize all reviews for each business
tokenized_reviews = {}
for business_id, reviews in filtered_reviews.items():
  #Store the tokenized reviews into a dictionary
    tokenized_reviews[business_id] = [tokenize_review(review) for review in reviews]

In [34]:
# Combine all tokens into a single list
all_tokens = [token for reviews in tokenized_reviews.values() for review in reviews for token in review]

In [35]:
# print the tokens, every 10 tokens in a new line
print("printing unigrams :")
for i in range(0, len(all_tokens), 10):
    print(all_tokens[i:i+10])

Streaming output truncated to the last 5000 lines.
['best', 'very', 'kind', 'and', 'friendly', 'and', 'honest', 'about', 'what', 'needs']
['to', 'be', 'done', 'your', 'plumber', 'joe', 'was', 'very', 'efficient', 'and']
['pleasant', 'however', 'i', 'thought', 'that', 'the', 'charge', 'for', 'replacing', 'the']
['shut', 'off', 'value', 'was', 'excessive', 'since', 'the', 'toilet', 'had', 'to']
['be', 'removed', 'in', 'order', 'to', 'replace', 'the', 'wax', 'seal', 'anyhow']
['peace', 'paul', 'kelly', 'i', 'have', 'used', 'bill', 'metzger', 'plumbing', 'for']
['both', 'for', 'personal', 'and', 'for', 'business', 'the', 'reason', 'why', 'i']
['call', 'them', 'everytime', 'is', 'because', 'they', 'have', 'always', 'got', 'someone']
['out', 'that', 'same', 'day', 'for', 'me', 'they', 'are', 'professional', 'and']
['great', 'communicators', 'call', 'bill', 'metzger', 'plumbing', 'for', 'any', 'plumbing', 'issues']
['outstanding', 'work', 'appearance', 'and', 'scheduling', 'they', 'are', 'the



The above operation results in a dictionary with business IDs (PIDs) representing keys and a list of tokenized reviews for each business.

At this stage, all reviews for each PID are tokenized and are stored as a value in the new dictionary.

This tokenization process breaks down each review into individual words or tokens, allowing for further analysis and processing.
The regular expression used allows for words with hyphens and apostrophes, which is important for preserving certain meaningful phrases.

Next, we'll use these tokenized reviews to identify important bigrams and create a more comprehensive vocabulary.

<div class="alert alert-block alert-warning">
    
### 4.2. Finding Bigrams and Calculating the vocabulary <a class="anchor" name="whetev"></a>

In this section, we've enhanced our tokenization by identifying and incorporating important bigrams:

1. We used NLTK's collocation finder to identify the top 200 bigrams based on PMI.
2. These bigrams were then used to create a Multi-Word Expression (MWE) tokenizer.
3. We applied this MWE tokenizer to our previously tokenized reviews, effectively treating important bigrams as single tokens.
4. Finally, we created a vocabulary that includes both unigrams and these important bigrams.

This approach allows us to capture meaningful two-word phrases while still maintaining individual words,
providing a richer representation of the language used in the reviews.

The resulting vocabulary gives us insight into the most common and important terms used across all reviews,
including both single words and key phrases.

In [36]:
# Set the bigram measure
bigram_measures = nltk.collocations.BigramAssocMeasures()
#Find bigram in the tokens
finder = nltk.collocations.BigramCollocationFinder.from_words(all_tokens)
finder.apply_freq_filter(5)
#Select the top 200 bigram based on the PMI
top_200_bigrams = finder.nbest(bigram_measures.pmi, 200)

# Create MWETokenizer with the top 200 bigrams
mwe_tokenizer = MWETokenizer(top_200_bigrams)

print(" top 200 bigrams present :")
print(top_200_bigrams)

 top 200 bigrams present :
[('pok', 'mon'), ('all-you', 'can-eat'), ('edward', 'niebla'), ('citrus', 'heights'), ('wells', 'fargo'), ('h.o', 'p.e'), ('hash', 'browns'), ('tyler', 'mackay'), ('carne', 'asada'), ('gen', 'kai'), ('laundered', 'mutt'), ('brett', 'guidry'), ('pot', 'roast'), ('nayan', 'pandey'), ('walnut', 'creek'), ('kyle', 'wilson'), ('timing', 'belt'), ('copper', 'beech'), ('pig', 'candy'), ('horseback', 'riding'), ('garbage', 'disposal'), ('cypress', 'veterinary'), ('veterinary', 'hospital'), ('parkway', 'plaza'), ('santa', 'rosa'), ('shoe', 'covers'), ('platinum', 'triangle'), ("lenny's", 'smokehouse'), ('los', 'angeles'), ('wedding', 'band'), ('tire', 'rotation'), ('anthony', 'mendoza'), ('huntington', 'beach'), ('wynn', 'vuong'), ('mashed', 'potatoes'), ('tennis', 'courts'), ('jefferson', 'platinum'), ('god', 'bless'), ('mango', 'habanero'), ('floral', 'studio'), ('buffalo', 'wild'), ('morgan', 'hill'), ('jordan', 'brown'), ('soda', 'stream'), ('horse', 'stables'), (

In [37]:
# Function to apply MWETokenizer to a single review
def apply_mwe_tokenizer(review_tokens):
    return mwe_tokenizer.tokenize(review_tokens)

# Apply MWETokenizer to all reviews this ensures the bigram is found within a review
mwe_tokenized_reviews = {}
for business_id, reviews in tokenized_reviews.items():
    mwe_tokenized_reviews[business_id] = [apply_mwe_tokenizer(review) for review in reviews]


In [38]:
# Flatten all tokens including bigrams
all_tokens_with_bigrams = [token for reviews in mwe_tokenized_reviews.values() for review in reviews for token in review]

print("All tokens with bigrams:")
for i in range(0, len(all_tokens_with_bigrams), 10):
    print(all_tokens_with_bigrams[i:i+10])

Streaming output truncated to the last 5000 lines.
['i', 'consider', 'the', 'installation', 'was', 'not', 'initially', 'completed', 'by', 'metzger']
['i', 'have', 'been', 'a', 'loyal', 'customer', 'for', 'years', 'and', 'this']
['is', 'the', 'first', 'instance', 'in', 'which', 'i', 'was', 'disappointed', 'in']
['the', 'service', 'luis', 'was', 'a', 'perfect', 'technician', 'and', 'did', 'a']
['great', 'job', 'and', 'in', 'a', 'timely_manner', 'installing', 'my', 'water', 'heater']
['on', 'a', 'saturday', 'luis', 'was', 'professional', 'and', 'knowledgeable', 'we', 'called']
['on', 'a', 'friday', 'afternoon', 'and', 'metzger', 'was', 'at', 'our', 'door']
['in', 'minutes', 'he', 'quickly', 'resolved', 'our', 'issue', 'and', 'was', 'on']
['his', 'way', 'nice', 'guy', 'and', 'a', 'hard', 'worker', 'found', 'these']
['guys', 'on', 'best', 'plumbers', 'a', 'really', 'friendly', 'gentleman', 'named', 'luis']
['arrived', 'and', 'saved', 'the', 'day', 'this', 'guy', 'really', 'knows', 'water']


In [39]:
# Create vocabulary
vocab = set(all_tokens_with_bigrams)

# Print sample of vocabulary
print("Sample of vocabulary (including bigrams):")
print(list(vocab)[:20])

# Print some statistics
print(f"Total unique tokens in vocabulary: {len(vocab)}")
print(f"Number of bigrams found in reviews: {sum(1 for token in vocab if '_' in token)}")

Sample of vocabulary (including bigrams):
['decide', 'helpfull', 'outline', 'veganized', 'spraying', 'fountain', 'tj', 'sufficient', 'cory', 'bentfor', 'brink', 'suppose', 'moisture', 'andreas', 'grabbed', 'guard', 'grill', 'somewhere', 'ho-down', 'blood']
Total unique tokens in vocabulary: 16954
Number of bigrams found in reviews: 198


In the section below, we refine our vocabulary by removing stop words, rare tokens, and applying stemming.
This process helps to focus on the most meaningful words and reduce the dimensionality of our data.

In [41]:

# Load stop words
with open('/content/drive/Shareddrives/FIT5196_S2_2024/GroupAssessment1/stopwords_en.txt', 'r') as f:
    stop_words = set(f.read().splitlines())

# Calculate vocabulary containing both unigrams and bigrams
vocab = set(all_tokens_with_bigrams)

# Count occurrences of each token across businesses
token_business_count = defaultdict(set)
for business_id, reviews in mwe_tokenized_reviews.items():
    for review in reviews:
        for token in set(review):
           # Use set to count unique occurrences per business
            token_business_count[token].add(business_id)

# Calculate total number of businesses
total_businesses = len(mwe_tokenized_reviews)

# Remove context-independent stop words
stopped_tokens = [w for w in vocab if w not in stop_words]

print(f"The number of tokens before removing stopwords is {len(vocab)}")
print(f"The number of tokens after removing stopwords is {len(stopped_tokens)}")
diff_tokens = set(vocab) - set(stopped_tokens)
print(f"The different tokens are {diff_tokens}")

The number of tokens before removing stopwords is 16954
The number of tokens after removing stopwords is 16430
The different tokens are {'other', 'which', 'me', 'd', 'associated', 'certainly', 'here', 'former', 'indeed', 'have', "couldn't", "you've", 'therefore', 'somewhere', 'appreciate', 'since', 'becomes', 'otherwise', 'ignored', 'happens', 'example', 'inner', 'currently', 'say', 'toward', 'somewhat', 'follows', 'ex', 'useful', 'really', 'seven', 'because', 'sure', 'either', 'thanks', 'y', 'p', 'near', 'meanwhile', 'namely', 'onto', 'asking', 'th', 'and', 'until', 'myself', 'whose', 'says', 'considering', "you'll", 'brief', 'm', "c'mon", 'right', 'specified', 'could', 'beside', 'behind', 'saying', 'consequently', 'after', 'except', 'wish', 'fifth', 'this', 'seeing', 'followed', 'l', "hasn't", 'soon', 'tell', 'off', 'has', 'sometimes', 'shall', 'looks', 'even', 'neither', 'above', 'than', 'want', 'particularly', 'to', 'enough', 'selves', 'no', 'who', 'welcome', 'w', 'twice', 'accordi

In [42]:
# Remove context-dependent stop words (appear in more than 95% of businesses)
stopped_ctokens = {token for token in stopped_tokens if len(token_business_count[token]) <= 0.95 * total_businesses}

print(f"The number of tokens before removing context-dependent stopwords is {len(stopped_tokens)}")
print(f"The number of tokens after removing context-dependent stopwords is {len(stopped_ctokens)}")
diff_tokens = set(stopped_tokens) - set(stopped_ctokens)
print(f"The different tokens are {diff_tokens}")

The number of tokens before removing context-dependent stopwords is 16430
The number of tokens after removing context-dependent stopwords is 16417
The different tokens are {'made', 'back', 'place', 'time', 'friendly', 'service', 'good', 'recommend', 'excellent', 'people', 'great', 'nice', 'awesome'}


In [43]:
# Remove rare tokens (appear in less than 5% of businesses)
stopped_rtokens = {token for token in stopped_ctokens if len(token_business_count[token]) >= 0.05 * total_businesses}

print(f"The number of tokens before removing rare tokens is {len(stopped_ctokens)}")
print(f"The number of tokens after removing rare tokens is {len(stopped_rtokens)}")
diff_tokens = set(stopped_ctokens) - set(stopped_rtokens)
print(f"The different tokens are {diff_tokens}")

The number of tokens before removing rare tokens is 16417
The number of tokens after removing rare tokens is 3332
The different tokens are {'helpfull', 'spraying', 'tj', 'cory', 'bentfor', 'brink', 'moisture', 'andreas', 'ho-down', 'trees', 'professiona', 'gwacamoly', 'spurs', 'income', 'contradictory', 'consulted', 'mouthwatering', 'harry', 'takeouts', 'qiuckly', 'ext', 'uniform', 'underwears', 'rudly', 'profile', 'gay', 'poea', 'reminders', 'teaming', 'florals', 'sag', 'speciality', 'stage', 'apron', 'sparing', 'resealed', 'watch.fair', 'passes', 'crumble', 'grown', 'blinked', 'tx', 'pureza', 'pup', 'yown', 'business.now', 'elevation', 'regencies', 'entails', 'recommed', 'og', 'stitches', 'wv', 'veterinarians', 'misled', 'oak', 'boothtooth', 'msrp', 'eventhou', 'riffert', 'ranchero', 'identified', 'lifesaving', 'unmatched', 'raining', 'ridicules', 'aquariam', 'shabby', 'swab', 'contemplation', 'consultative', 'ooomy', 'abusive', 'counters', 'suffering', 'cremo', 'groomer', 'sights', 

In [44]:
# Remove tokens with length less than 3
stopped_rtokens_3 = {token for token in stopped_rtokens if len(token) >= 3}

print(f"The number of tokens before removing tokens less than 3 length is {len(stopped_rtokens)}")
print(f"The number of tokens after removing tokens less than 3 length is {len(stopped_rtokens_3)}")
diff_tokens = set(stopped_rtokens) - set(stopped_rtokens_3)
print(f"The different tokens are {diff_tokens}")


The number of tokens before removing tokens less than 3 length is 3332
The number of tokens after removing tokens less than 3 length is 3299
The different tokens are {'ny', 'yo', 'la', 'el', 'da', 'id', 'st', 'bc', 'sf', 'ms', 'se', 'oz', 'ya', 'dr', 'pm', 'll', 'ft', 'yr', 'er', 'tv', 'al', 'ca', 'bs', 'ac', 'im', 'mr', 'de', 've', 'hr', 'em', 'en', 'ad', 'ur'}


In [45]:
# Apply Porter stemming to group words with same meaning
stemmer = PorterStemmer()
vocab = {stemmer.stem(token) for token in stopped_rtokens_3}

print(f"The number of tokens before stemming is {len(stopped_rtokens_3)}")
print(f"The number of tokens after stemming  is {len(vocab)}")

The number of tokens before stemming is 3299
The number of tokens after stemming  is 2301


Explanation of the vocabulary refinement process:

1. Stop words removal:
   - We first remove common English stop words (e.g., "the", "a", "an") that don't carry significant meaning.
   - We then remove context-dependent stop words that appear in more than 95% of businesses, as these are likely not distinctive.

2. Rare token removal:
   - Tokens appearing in less than 5% of businesses are removed to focus on more common and potentially more meaningful terms.

3. Short token removal:
   - Tokens with less than 3 characters are removed as they are often not meaningful on their own.

4. Stemming:
   - We apply Porter stemming to reduce words to their root form, helping to consolidate similar words.

This refined vocabulary will be more focused and meaningful for our analysis, reducing noise and improving the efficiency of our model.

<div class="alert alert-block alert-success">
    
## 5. Writing Output Files <a class="anchor" name="write"></a>

files need to be generated:
* Vocabulary list
* Sparse matrix (count_vectors)

This is performed in the following sections.

<div class="alert alert-block alert-warning">
    
### 5.1. Vocabulary List <a class="anchor" name="write-vocab"></a>

We create a file with unigram and bigram, each token is followed by its corresponding index obtained after sorting the token in alphabetical order.

List of vocabulary should also be written to a file, sorted alphabetically, with their reference codes in front of them. This file also refers to the sparse matrix in the next file. For this purpose, we create a sorted vocabulary and write it to a file.

In [46]:
sorted_vocab = sorted(vocab)
# Create a dictionary mapping tokens to their index in the vocabulary
vocab_index = {token: idx for idx, token in enumerate(sorted_vocab)}

# Write vocabulary to file
with open('103_vocab.txt', 'w', encoding='utf-8') as vocab_file:
    for token, index in vocab_index.items():
        vocab_file.write(f"{token}:{index}\n")

print("Vocabulary has been written to 103_vocab.txt")

Vocabulary has been written to 103_vocab.txt


<div class="alert alert-block alert-warning">
    
### 5.2. Sparse Matrix <a class="anchor" name="write-sparseMat"></a>

For writing sparse matrix for a paper, we firstly calculate the frequency of words for that business.
This creates a compact representation of the reviews for each business.

In [47]:
# Initialize the stemmer
stemmer = PorterStemmer()

# Load the vocabulary with indices
with open('103_vocab.txt', 'r', encoding='utf-8') as vocab_file:
    vocab_index = {line.split(':')[0]: int(line.split(':')[1].strip()) for line in vocab_file}

# Initialize dictionary to hold sparse representations
sparse_representation = {}

# Process each business's reviews
for business_id, reviews in mwe_tokenized_reviews.items():
    # Flatten all reviews for this business into a single list of tokens and stem them
    all_tokens = [stemmer.stem(token) for review in reviews for token in review]

    # Create frequency distribution
    freq_dist = FreqDist(all_tokens)

    # Convert the frequency distribution to the required  representation
    sparse_vector = []
    for word, freq in freq_dist.items():
        if word in vocab_index:
          #Display as token index : token frequency
            sparse_vector.append(f"{vocab_index[word]}:{freq}")
     # Only include businesses with non-empty vectors
    if sparse_vector:
        sparse_representation[business_id] = ', '.join(sparse_vector)

# Write the sparse representation to countvec.txt
with open('103_countvec.txt', 'w', encoding='utf-8') as countvec_file:
    for business_id, sparse_vector_str in sparse_representation.items():
        countvec_file.write(f"{business_id}, {sparse_vector_str}\n")

print("Count vectors have been written to 103_countvec.txt")

Count vectors have been written to 103_countvec.txt


The generated files:
1. vocab.txt: Contains each token and its corresponding index.
2. countvec.txt: Contains the sparse representation of reviews for each business,
   where each line represents a business and its token counts using the vocabulary indices.

These files provide a compact and efficient representation of the processed review data,
suitable for various text analysis and machine learning tasks.

-------------------------------------

<div class="alert alert-block alert-success">
    
## 6. Summary <a class="anchor" name="summary"></a>

In this notebook, we have successfully completed the following tasks:
1. Loaded and preprocessed the Google Maps review data
2. Tokenized the reviews and identified important bigrams
3. Refined the vocabulary by removing stop words and applying stemming
4. Created output files (vocab.txt and countvec.txt) for further analysis

These steps have transformed the raw review data into a format suitable for various NLP tasks such as sentiment analysis, topic modeling, or business classification.

-------------------------------------

<div class="alert alert-block alert-success">
    
## 7. References <a class="anchor" name="Ref"></a>

[1] More simplified explanation of chain.from_iterable and chain() of itertools,https://stackoverflow.com/questions/43151271/more-simplified-explanation-of-chain-from-iterable-and-chain-of-itertools#:~:text=8%2C%209%5D%20.-,chain.,for%20a%20flattening%20nested%20iterables., Accessed 26/08/2024.

[2]How removing rare words affects model performance? https://stackoverflow.com/questions/74074443/how-removing-rare-words-affects-model-performance Accessed  22/08/2024.



## --------------------------------------------------------------------------------------------------------------------------